In [1]:
import os
import scanpy as sc
import numpy as np
import rpy2.robjects as ro

os.environ["R_HOME"] = "/opt/homebrew/Cellar/r/4.5.2/lib/R"
os.environ["R_LIBS_USER"] = "/opt/homebrew/lib/R/library"
os.environ["DYLD_FALLBACK_LIBRARY_PATH"] = "/opt/homebrew/Cellar/r/4.5.2/lib/R/lib"

outdir = "../../../write/03_milo_DA/PICA0001-PICA0007"
os.makedirs(outdir, exist_ok=True)
sc.settings.figdir = "../../../figures/PICA0001-PICA0007/03_milo_DA"

Unable to determine R home: [Errno 2] No such file or directory: 'R'


RuntimeError: Unable to determine R_HOME.

In [2]:
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

: 

In [ ]:
adata = sc.read_h5ad("../../../write/03_batch_expression/PICA0001-PICA0007/PICA0001-PICA0007_combined_annot_with_age_scvi.h5ad")
adata

/Users/jessica/mambaforge/envs/scanpy/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 31769 × 38606
    obs: 'status', 'assignment', 'pica_id', 'pool_id', 'sequencing_batch', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'status_manual', 'cell_type', 'status_manual_01_b_subset', 'broad_cell_type', 'cell_type_01_b_cell', 'cell_type_01_b_cell_corrected', 'cell_type_02_t_cell_broad', 'cell_type_03_t_cell_subtype', 'cell_type_04_myeloid_subset', 'dataset_source', '_scvi_batch', '_scvi_labels', 'leiden_res_0.5', 'leiden_res_1.0', 'leiden_res_1.5', 'leiden_res_2.0', 'cell_type_cd4_res_1', 'origin', 'Event Name', 'Was baseline blood sample collected?', 'IFC ID', "Child's sex", 'Age in years at t

: 

In [ ]:
adata_no_knnx = adata.copy()

In [16]:
# recompute the neighborhood
sc.pp.neighbors(adata_no_knnx, use_rep="X_scVI")
adata_no_knnx.write_h5ad( "../../../write/03_milo_DA/PICA0001-PICA0007/PICA0001-PICA0007_milo_input.h5ad", compression="gzip")


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/jessica/mambaforge/envs/scanpy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
adata_r = adata_no_knnx.copy()
# Save the connectivity matrix
knn_adjacencyx = adata_r.obsp["connectivities"]
# pop the rest to make it work
adata_r.uns.pop("neighbors")
# adata_no_knnx.obsm.pop("_scvi_extra_continuous")   
adata_r.obsp = None

In [ ]:
%%R -i adata_r -i knn_adjacencyx
save(adata_r, knn_adjacencyx, file = '../../../write/03_milo_DA/PICA0001-PICA0007/PICA0001-PICA0007__milo_prep.RData')